<a href="https://colab.research.google.com/github/beekal/MachieneLearningProjects/blob/master/0%20Basics%20-%20TF/TF_2_0_Basics%20Revision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF 2.0 Topics Covered
1. TF.Data : A single point of entry to handle any/ varied data type ranging from pandas, csv, image, text, TfRecordByte e.t.c
2. TFX: Tensorflow extended,  which aims to  provided end to end TF ecosystem from its  research/ prototype to  the server deployment. It includes the following  all of  which we will cover in this notebook
  -  TF validation : Used to validate the data
  -  TF Transform : Used to transform the data to its modeling state
  -  TF Modeling / Analysis : Used to create the model and evaluate its performance. Reiterate until satisfactory Metric reached.
  - TF Serving: Used to serve the model to production with versioning/ Rollback capabilities.
   

In [8]:
import tensorflow as tf
import numpy

print(f'TF version : ', tf.__version__)

TF version :  2.2.0-rc3


## TF.Data
Depending on the data source you wil have to use different TF data load calls
  - Data in memory :
      - tf.data.Dataset.from_tensors() or
      - tf.data.Dataset.from_tensor_slices()

In [10]:
d_mem = tf.data.Dataset.from_tensor_slices([1,2])
print(f' Tf.Data from memory : ' ,[e.numpy() for e in d_mem])

 Tf.Data from memory :  [1, 2]


## TF Transform :
We can apply different type of transformations